## 1 Description

For this assignment, you will build a Sarsa agent which will learn policies in the OpenAI Gym [Frozen Lake](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py) environment. [OpenAI Gym](http://gym.openai.com/docs/) is a platform where users can test their RL algorithms on a selection of carefully crafted environments.

Frozen Lake is a grid world environment that is highly stochastic, where the agent must cross a slippery frozen lake which has deadly holes to fall through. The agent begins in the starting state $S$ and is given a reward of 1 if it reaches the goal state $G$. A reward of 0 is given for all other transitions.

The agent can take one of four possible moves at each state (left, down, right, or up). The frozen cells $F$ are slippery, so the agent’s actions succeed only $\frac{1}{3}$ of the time, while the other $\frac{2}{3}$ are split evenly between the two directions orthogonal to the intended direction. If the agent lands in a hole $H$, then the episode terminates.

You will be given a randomized Frozen Lake map with a corresponding set of parameters to train your Sarsa agent with.  
If your agent is implemented correctly, then after training it for the specified number of episodes, 
your agent will produce the same policy (not necessarily an optimal policy).

<img src="https://www.dropbox.com/s/cykbkk5mg5652n9/frozen_lake.png?raw=1" width=200px;>

Figure 1: Example Frozen Lake Map

The toturials of Fronzen_Lake.   [Frozen Lake env](https://reinforcement-learning4.fun/2019/06/16/gym-tutorial-frozen-lake/), [gym](https://gym.openai.com/envs/FrozenLake-v0/)

## 2 Sarsa $(S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1})$

Sarsa uses temporal-difference learning to form a model-free on-policy reinforcement-learning algorithm that solves the \textit{control} problem.  It is model-free because it does not need and does not use a model of the environment, namely neither a transition nor reward function; 
instead, Sarsa samples transitions and rewards online.

It is on-policy because it learns about the same policy that generates its behaviors (this is in contrast to Q-learning, which you'll examine in your next homework).  That is, Sarsa estimates the action-value function of its behavior policy. In this homework, you will not be training a Sarsa agent 
to approximate the \textit{optimal} action-value function; instead, the hyperparameters of both the 
exploration strategy and the algorithm will be given to you as input\textemdash the goal being to verify that your SARSA agent is correctly implemented.

## 3 Procedure

Since this homework requires you to match a non-deterministic output between your
agent and the autograder's agent, attention to detail to each of the following points 
is required:

- You must use Python and the library NumPy for this homework
- Install OpenAI Gym (e.g. `pip install gym`).
- Instantiate the Frozen Lake environment with `gym.make(FrozenLake-v0, desc=desc).unwrapped`, where the attribute desc is a square version of the string input **amap**. This means you must resize **amap** so that it looks like a square grid, where the input characters fill in the grid row by row.  Make sure you use the unwrapped method as indicated above.
- The input **amap** should be used to seed the random number generators for both Gym and NumPy.  Do *not* use the Python standard library's random library.
- Implement your Sarsa agent using an $\epsilon$-greedy behavioral policy.  Specifically, you must use `numpy.random.random` to choose whether or not the action is greedy, and `numpy.random.randint` to select the random action.
- Initialize the agent’s Q-table to zeros.
- Train your agent using the given input parameters.  The input `amap` is the Frozen Lake map that you need to resize and provide to the `desc` attribute when you instantiate your environment.  The input `gamma` is the discount rate. The input `alpha` is the learning rate.  The input `epsilon` is the parameter for the $\epsilon$-greedy behavior strategy your Sarsa agent will use.  Specifically, an action should be selected uniformly at random if a random number drawn uniformly between $0$ and $1$ is less than $\epsilon$.  If the greedy action is selected, the action with lowest index should be selected in case of ties.  The input `n_episodes` is the number of episodes to train your agent.  Finally, `seed` is the number used to seed both Gym's random number generator `and` NumPy's random number generator.
- To sync with the autograder, your Sarsa implementation should select the action corresponding to the next state the agent will visit *even when* that next state is a terminal state (this action will never be executed by the agent).
- The answer you provide should be the greedy policy with respect to the Q-function obtained by your Sarsa agent after the completion of the final episode. Specifically, the policy should be expressed as a string of characters: `<, v, >, ^`, representing left, down, right, and up, respectively.  The ordering of the actions in the output should reflect the ordering of states in **amap**.  Provide answers for the specific problems you are given on Canvas.

## 4 Examples

The following examples can be used to verify that your agent is implemented correctly.
- Input: `amap="SFFG"; gamma=1.0; alpha=0.24; epsilon=0.09; n_episodes=49553; seed=202404`
    
  Output: `<<v<`
  
- Input: `amap="SFFFHFFFFFFFFFFG"; gamma=1.0; alpha=0.25; epsilon=0.29; n_episodes=14697; seed=741684`
    
  Output: `^vv><>>vvv>v>>><`

- Input: `amap="SFFFFHFFFFFFFFFFFFFFFFFFG"; gamma=0.91; alpha=0.12; epsilon=0.13; n_episodes=42271; seed=983459`
    
  Output: `^>>>><>>>vvv>>vv>>>>v>>^<`

## 5 Implementation

We provide some start code for reference. Feel free to modify it. You are required to finish the TODO parts to let the agent run.

In [1]:
!pip install gymnasium[classic_control]

# Environment
import gymnasium as gym
import numpy as np
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="ansi")

# initialzation
MOVES = {
    0: "<",
    1: "v",
    2: ">",
    3: "^",
}

In [2]:
# Agent

def solve(env, gamma, alpha, epsilon, n_episodes, seed):
    """
    Args:
        env: Fronzen_Lake environment. See Fronzen Lake env link in section 1 about toturials of Fronzen_Lake.
        gamma: discount rate
        alpha : learning rate
        epsilon : parameter for the  𝜖 -greedy behavior
        n_episodes : the number of episodes to train your agent
        seed : the number used to seed both Gym’s random number generator and NumPy’s random number generator
    Return:
        Don't need to return anything, just fill out the TODOs.
    """
    # Note: please see how to define the env is the following Test Example.

    # TODO: Set seed
    # Hint: use np.random.seed and env.seed
    np.random.seed(seed)
    env.action_space.seed(seed)
    
    # TODO: Initialize the agent’s Q-table to zeros. 
    # Hint: use np.zeros with env.observation_space.n and env.action_space.n
    Q = np.zeros((env.observation_space.n, env.action_space.n))

    for i in range(n_episodes):
        # define state
        state, _ = env.reset(seed=seed)
        done = False
        # TODO: define action
        # consider two cases: np.random.random() > epsilon or not
        if np.random.random() > epsilon:
            # Exploitation: choose best action
            action = np.argmax(Q[state,:])
        else:
            # Exploration: pick random action
            action = np.random.randint(0, 4)
        
        # Update state and action
        while not done:
            # TODO: get new_state, reward, done 
            # Hint: use env.step()
            new_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            # TODO: get new_action
            if np.random.random() > epsilon:
                new_action = np.argmax(Q[new_state])
            else:
                new_action = np.random.randint(0, 4)
                
            # TODO: Q-function
            Q[state, action] += alpha * (reward + gamma + Q[new_state, new_action] - Q[state, action])

            # update state and action
            state = new_state
            action = new_action
    # print the result        
    print(",".join([MOVES[np.argmax(i)] for i in Q]))

In [4]:
# Test Example
amap = "SFFG"
amap_row = 2
amap_col = 2

env = gym.make("FrozenLake-v1",
    desc = np.reshape(list(amap), (amap_row,amap_col)),
    map_name=None,
    is_slippery=False,
    render_mode="ansi"
).unwrapped

for idx, row in enumerate(env.unwrapped.desc):
    print(f"Row {idx}: {''.join(row.astype(str))}")

seed=202404
gamma=1.0
alpha=0.24
epsilon=0.09
n_episodes=49553

solve(env, gamma, alpha, epsilon, n_episodes, seed)

# expected output: <,<,v,<

Row 0: SF
Row 1: FG
^,<,^,<
